In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException



import openpyxl
import os
from bs4 import BeautifulSoup as soup
import pandas as pd
import time
import shutil

In [2]:
!export PATH=$PATH:/home/dmacs/Documents/Dissertation_Codes/Data_Acquisition/geckodriverlinux64/geckodriver

In [3]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

#Accepting SSL certificates
capabilities = webdriver.DesiredCapabilities().FIREFOX
capabilities['acceptSslCerts'] = True

#Open driver without opening window
options = Options()
options.add_argument('--headless')

In [4]:
old_path = '/home/dmacs/Documents/Pranav Phanindra/Data_Scraping_Final/Data_Part_2/Data_excel_2/'
new_path_1 = '/home/dmacs/Documents/Pranav Phanindra/Data_Scraping_Final/Data_Part_2/Data_excel_2/'
new_path_2 = '/home/dmacs/Documents/Pranav Phanindra/Data_Scraping_Final/Data_Part_2/Data_html_2/'


In [5]:
#Setting preferences to download directory
profile = webdriver.FirefoxProfile()
profile.set_preference('browser.download.folderList',2)
profile.set_preference('browser.download.manager.showWhenStarting',False)
profile.set_preference('browser.download.dir',
                           '/home/dmacs/Documents/Pranav Phanindra/Data_Scraping_Final/Data_Part_2/Data_excel_2/')
profile.set_preference('browser.helperApps.neverAsk.saveTODisk','application/x-gzip')



link = 'https://agmarknet.gov.in/SearchCmmMkt.aspx?Tx_Commodity=29&Tx_State=CG&Tx_District=0&Tx_Market=0&DateFrom=01-Apr-2008&DateTo=30-Apr-2019&Fr_Date=01-Apr-2008&To_Date=30-Apr-2019&Tx_Trend=0&Tx_CommodityHead=Barley+(Jau)&Tx_StateHead=Chattisgarh&Tx_DistrictHead=--Select--&Tx_MarketHead=--Select-- '
link_driver = webdriver.Firefox(capabilities=capabilities,firefox_profile = profile)
link_driver.get(link)


In [6]:
#Setting date to required financial year date
dateto = link_driver.find_element(By.XPATH ,"//*[@id='txtDateTo']" )
dateto.send_keys(Keys.CONTROL,'a')
dateto.send_keys(Keys.BACKSPACE)
dateto.send_keys('31-Mar-2022')
dateto.send_keys(Keys.ENTER)
time.sleep(5)

In [7]:
state = Select(link_driver.find_element(By.ID , "ddlState"))
state_options = state.options
#print(state_options)


commodity = Select(link_driver.find_element(By.ID , "ddlCommodity"))
commodity_options = commodity.options
#print(commodity_options)


exceptions = (StaleElementReferenceException,NoSuchElementException)

In [8]:
for commodity_i in commodity_options :
    print(commodity_i.get_attribute("text"))
        

--Select--
Ajwan
Alasande Gram
Almond(Badam)
Alsandikai
Amaranthus
Ambada Seed
Amla(Nelli Kai)
Amphophalus
Antawala
Anthorium
Apple
Apricot(Jardalu/Khumani)
Arecanut(Betelnut/Supari)
Arhar (Tur/Red Gram)(Whole)
Arhar Dal(Tur Dal)
Ashgourd
Astera
Avare Dal
Bajra(Pearl Millet/Cumbu)
Balekai
Bamboo
Banana
Banana - Green
Barley (Jau)
Bay leaf (Tejpatta)
Beans
Beaten Rice
Beetroot
Bengal Gram Dal (Chana Dal)
Bengal Gram(Gram)(Whole)
Ber(Zizyphus/Borehannu)
Betal Leaves
Betelnuts
Bhindi(Ladies Finger)
Big Gram
Binoula
Bitter gourd
Black Gram (Urd Beans)(Whole)
Black Gram Dal (Urd Dal)
Black pepper
BOP
Borehannu
Bottle gourd
Bran
Brinjal
Broken Rice
Broomstick(Flower Broom)
Bull
Bullar
Bunch Beans
Butter
Cabbage
Calf
Camel Hair
Cane
Capsicum
Cardamoms
Carnation
Carrot
Cashew Kernnel
Cashewnuts
Castor Oil
Castor Seed
Cauliflower
Chakotha
Chapparad Avare
Chennangi (Whole)
Chennangi Dal
Cherry
Chikoos(Sapota)
Chili Red
Chilly Capsicum
Chow Chow
Chrysanthemum
Chrysanthemum(Loose)
Cinamon(Dalchini

In [9]:
states_array = []
for state_i in state_options[1:] :
    print(state_i.get_attribute("text"))
    states_array.append(state_i.get_attribute("text"))

Andaman and Nicobar
Andhra Pradesh
Arunachal Pradesh
Assam
Bihar
Chandigarh
Chattisgarh
Dadra and Nagar Haveli
Daman and Diu
Goa
Gujarat
Haryana
Himachal Pradesh
Jammu and Kashmir
Jharkhand
Karnataka
Kerala
Lakshadweep
Madhya Pradesh
Maharashtra
Manipur
Meghalaya
Mizoram
Nagaland
NCT of Delhi
Odisha
Pondicherry
Punjab
Rajasthan
Sikkim
Tamil Nadu
Telangana
Tripura
Uttar Pradesh
Uttrakhand
West Bengal


In [11]:
"""
for index in range(1, len(state_options)+1):
    state.select_by_index(index)
    """
commodities = ['Maize','Barley (Jau)','Wheat','Bengal Gram(Gram)(Whole)',
               'Bengal Gram Dal (Chana Dal)','Soyabean','Mustard']

element = WebDriverWait(link_driver,20,ignored_exceptions=exceptions) 

for commodity_i in commodities[1:]:
    element.until(EC.presence_of_element_located((By.ID, "ddlCommodity")))
    element.until(EC.element_to_be_clickable((By.ID, "ddlCommodity")))
    commodity = Select(link_driver.find_element(By.ID , "ddlCommodity"))
    commodity.select_by_visible_text(commodity_i)
    
    for state_i in states_array:
        element.until(EC.presence_of_element_located((By.ID, "ddlState")))
        state = Select(link_driver.find_element(By.ID , "ddlState"))
        state.select_by_visible_text(state_i)
        
        
        try : 
            #Wait until the required element is seen 
            go_button = link_driver.find_element(By.XPATH,"//*[@id='btnGo']")
            element.until(EC.presence_of_element_located((By.ID,"btnGo")))
            element.until(EC.element_to_be_clickable((By.ID,"btnGo")))
            time.sleep(10)
            go_button.click()
            time.sleep(10)
            
            
            element.until(EC.presence_of_element_located((By.ID, "cphBody_ButtonExcel")))
            to_Excel = link_driver.find_element(By.XPATH , "//*[@id='cphBody_ButtonExcel']")
            to_Excel.click()
            time.sleep(20)
            
            link_driver.refresh()
            
            dateto = link_driver.find_element(By.XPATH ,"//*[@id='txtDateTo']" )
            dateto.send_keys(Keys.CONTROL,'a')
            dateto.send_keys(Keys.BACKSPACE)
            dateto.send_keys('31-Mar-2022')
            dateto.send_keys(Keys.ENTER)
            time.sleep(5)
            
            
            
            
            
            
            state_index = link_driver.find_element(By.XPATH , "//*[@id='ddlState']").get_attribute(name='value')
            commodity_index = link_driver.find_element(By.ID , 'ddlCommodity').get_attribute(name='value')
            
            old_name =  old_path + 'Agmarknet_Price_Report.xls'
            new_name_1 = new_path_1 + '{}-{}.xls'.format(commodity_index,state_index)
            new_name_2 = new_path_2 + '{}_{}.html'.format(commodity_index,state_index)
            
            
            
            if 'Agmarknet_Price_Report.xls' in os.listdir(old_path) :
                shutil.copy(old_name, new_name_1)
                os.rename(old_name, new_name_2)
        
        
        finally :
            print('done')
    

done


StaleElementReferenceException: Message: The element reference of <input id="btnGo" class="btn" name="ctl00$btnGo" type="submit"> is stale; either the element is no longer attached to the DOM, it is not in the current frame context, or the document has been refreshed
Stacktrace:
WebDriverError@chrome://remote/content/shared/webdriver/Errors.jsm:188:5
StaleElementReferenceError@chrome://remote/content/shared/webdriver/Errors.jsm:469:5
element.resolveElement@chrome://remote/content/marionette/element.js:691:11
evaluate.fromJSON@chrome://remote/content/marionette/evaluate.js:262:31
evaluate.fromJSON@chrome://remote/content/marionette/evaluate.js:270:29
receiveMessage@chrome://remote/content/marionette/actors/MarionetteCommandsChild.jsm:81:34


In [ ]:
link_driver.quit()

In [ ]:
a == c